In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
import numpy as np
import json
from data_transfer_utility.application import DataTransferUtility
from xgboost import XGBClassifier
import shap
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import metrics
from lifelines.utils import concordance_index
import random
import copy
from cryptography.fernet import Fernet
import pickle
import datetime

import re
import os
from datetime import date,timedelta
import matplotlib.pyplot as plt

today_date = date.today()

In [ ]:
output_dir = f'data_backup/m2_mds_{today_date}/'
# output_dir = f'data_backup/m2_mds_2024-05-29/'
if output_dir.split('/')[1] not in os.listdir('data_backup/'):
    os.makedirs(output_dir)
text_file = open(output_dir+'output.txt','w')
# # text_file = open(output_dir+'output.txt','a')

In [ ]:
text_file.write("Consdiering onlly lab based features.I have added labs like wbc, hemoglobin, platelates,ferritin,blasts and neutrophil as features.\n\
Removed medication, imaging, stage, biomarker and labs like 489004, 26516-5, 53800-9, 74464-9,13056-7, 48725-6, 93846-4, 33256-9, 30350-3,804-5,49497-1,20509-6,30313-1,778-1,30351-1,49498-9\n\
Also removed Mean, Median, 25th percentile, 75th percentile, Range, Min delta, Max delta, Satandard Deviattion for lab stats.\n\
Considered mean of respectiive different code features belong to same lab group")

In [ ]:
from mds_features_model2 import *
from mds_labs_model2 import *

In [ ]:
REDSHIFT_CREDENTIALS = json.load(open('../secrets/dtu_redshift_credentials.json', 'r'))
dtu = DataTransferUtility(df_mode='pandas', **REDSHIFT_CREDENTIALS)

In [ ]:
# biomarker_schemas = {'Lung':'c3_gn360_202312_nsclc',
#                      'Breast':'c3_gn360_202309_breast',
#                      'Prostate':'c3_gn360_202309_prostate',
#                      'Bladder':'c3_gn360_202309_bladder',
#                      'Pancreas':'c3_gn360_202309_pancreatic',
#                      'Colorectal':'c3_gn360_202309_crc'}

biomarker_schemas= {'Bladder':'c3_gn360_202403_bladder',
                    'Breast':'c3_gn360_202403_breast',
                    'Colorectal':'c3_gn360_202403_crc',
                    'NSCLC':'c3_gn360_202403_nsclc',
                    'Ovarian':'c3_gn360_202403_ovarian',
                    'Pancreas':'c3_gn360_202403_pancreatic',
                    'Prostate':'c3_gn360_202403_prostate',
                    'SCLC':'c3_gn360_202403_sclc'}

In [ ]:
cohort = 'mds'
schema = 'cdm_structured_202402'
# index_date_table = 'model_factory_dev.ofpc_LoT_index_date_c3_cdm_ss_all_ovarian_202312_v2_2_1_20231208_0619'
initial_dx_table = 'model_factory_dev.initial_dx_date_cdm_structured_202402_v2_5_20240404_1034'
lot_table_name = 'line_of_therapy.mds_rbs_rwd360_cdm_structured_202402_lot_v6_7_20240404_1146'

In [ ]:
index_date_df = get_mds_initial_dx_table(initial_dx_table)
patient_ids = list(set(index_date_df['chai_patient_id']))
pat_str = "','".join(patient_ids)

In [ ]:
patient_test_df = dtu.redshift_to_df('''SELECT 
    chai_patient_id,curation_indicator,test_name_standard_name,test_name_standard_code,
    test_date,test_value_numeric_standard,test_unit_standard_name FROM {schema}.patient_test
    WHERE chai_patient_id IN ('{pat_str}')'''.format(schema=schema,pat_str=pat_str))

In [ ]:
patient_test_df.shape

In [ ]:
unique_labs = patient_test_df['test_name_standard_name'].unique()

In [ ]:
unique_labs

In [ ]:
blast_list = [x for x in unique_labs if 'Blasts' in x]

In [ ]:
temp_patient_test_df = patient_test_df[patient_test_df['test_name_standard_code'].isin(['30376-8','708-8'])]

In [ ]:
temp_patient_test_df['test_unit_standard_name'].value_counts()

In [ ]:
temp = patient_test_df[patient_test_df['test_name_standard_name'].apply(lambda x:'ecog' in x.lower())]
temp_lab = temp.groupby(['test_name_standard_name','test_name_standard_code']).count()
temp_lab = temp_lab.sort_values(by='chai_patient_id',ascending=False)

In [ ]:
temp_lab.shape

In [ ]:
temp_lab = temp_lab.reset_index()
temp_lab = temp_lab[['test_name_standard_name','test_name_standard_code','chai_patient_id']].rename(columns={'chai_patient_id':'count'})
temp_lab.head()

In [ ]:
temp_lab.to_csv('mds_ecog_counts.csv')

In [ ]:
from datetime import date
import re
import time
import datetime
from datetime import timedelta

def _identify_following_month_first_date(month, year):
    if month == 12:
        cutoff_year = int(year) + 1
        cutoff_month = 1
    else:
        cutoff_year = year
        cutoff_month = month + 1

    cut_off_date = datetime.datetime(
        int(cutoff_year), cutoff_month, 1
    )

    return cut_off_date


def prepare_cutoff_date(schema_name):
    current_year = str(date.today()).split("-")[0]
    prev_year = str(int(str(date.today()).split("-")[0]) - 1)
    if re.search(current_year + r"(\d+)", schema_name):
        year = current_year
    elif re.search(prev_year + r"(\d+)", schema_name):
        year = prev_year
    month = int(re.search(year + r"(\d+)", schema_name).group(1))
    cut_off_date = _identify_following_month_first_date(month, year)

    return (cut_off_date - timedelta(days=1))


In [ ]:
def main(schema, initial_dx_table, lot_table_name, cohort):
    feature_dict = {}
    cohort_org = copy.deepcopy(cohort)
    if cohort in ['NSCLC','SCLC']:
        cohort='Lung'
    # cond_table = get_condition_table(schema, cohort)
    # pat_ids = list(cond_table.unique())

    # index_date_df = get_index_table(index_date_table, cond_table)
    # index_date_df = get_index_table_wo_cond(index_date_table)
    index_date_df = get_mds_initial_dx_table(initial_dx_table)
    index_date_df = index_date_df.rename(columns={'diagnosis_date':'index_date'})
    index_date_df = index_date_df[index_date_df["index_date"].notnull()]
    print(f"Index_date : {index_date_df.shape}, unique patients : {index_date_df['chai_patient_id'].nunique()}")

    # cond_table = cond_table.merge(index_date_df[['chai_patient_id','index_date']],on=['chai_patient_id'],how='inner')
    cond_table = index_date_df[['chai_patient_id','index_date']]
    cond_table = cond_table[cond_table['index_date'].dt.year>2014]
    # cond_table = cond_table[cond_table['index_date']>cond_table['diagnosis_date']]
    cond_table = cond_table.drop_duplicates(subset=['chai_patient_id','index_date'])

    censored = get_censored_patients(schema, cond_table)
    cut_off_date = prepare_cutoff_date(schema)
    censored["cut_off_date"] = cut_off_date
    censored['last_available_date'] = censored.apply(lambda x:x['cut_off_date'] if (x['cut_off_date']<x['last_available_date']) else x['last_available_date'],axis=1)
    print(f"censored : {censored.shape}, unique patients : {censored['chai_patient_id'].nunique()}")

    cond_table =cond_table.merge(censored,on='chai_patient_id',how='left')
    cond_table['duration'] = cond_table.apply(lambda x:(x['last_available_date']-x['index_date']).days,axis=1)
    cond_table = cond_table[cond_table['duration']>0]
    print(f"condition : {cond_table.shape}, unique patients : {cond_table['chai_patient_id'].nunique()}")
    cond_table['random_point'] = cond_table.apply(
        lambda x: 0 if x['duration'] in [0, 1] else random.randint(1, x['duration'] - 1), axis=1)
    cond_table['random_date'] = cond_table.apply(lambda x:x['index_date']+timedelta(days=x['random_point']),axis=1)

    lot_table = get_lot_table(lot_table_name, cond_table=cond_table)
    cond_table = cond_table.merge(lot_table,on='chai_patient_id',how='left')
    
    cond_table['lot_diff'] = cond_table.apply(lambda x:(x['regimen_end']-x['random_date']).days,axis=1)
    cond_table['lot_diff'] = cond_table['lot_diff'].apply(lambda x:np.nan if x<=0 else x)
    cond_table = cond_table.sort_values(by=['lot_diff'],ascending=True)
    cond_table = cond_table.drop_duplicates(subset=['chai_patient_id'],keep='first')
    cond_table['event'] = cond_table['lot_diff'].apply(lambda x:0 if pd.isna(x) else 1)
    cond_table['TTE'] = cond_table.apply(lambda x:x['lot_diff'] if (x['event']==1) else (x['last_available_date']-x['random_date']).days,axis=1)

    cond_table.to_csv(f'{output_dir}cond_table_{cohort_org}_{today_date}.csv')
    # cond_table = pd.read_csv(f'data_backup/m2_mds_2024-05-29_with_biomarker/cond_table_{cohort_org}_2024-05-29.csv')
    # cond_table['random_date'] = cond_table['random_date'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d'))
    
    text_file.write(f"Cohort : {cohort_org}")
    text_file.write(f"Label value counts : \n{cond_table['event'].value_counts()}\n\n")
    text_file.write(f"Duration stats :\n")
    text_file.write(f"For event data points (event = 1) : \n{cond_table[cond_table['event']==1]['duration'].describe()}\n\n")
    text_file.write(f"For censor data points (event = 0) : \n{cond_table[cond_table['event']==0]['duration'].describe()}\n\n")
    text_file.write(f"TTE stats :\n")
    text_file.write(f"For event data points (event = 1) : \n{cond_table[cond_table['event']==1]['TTE'].describe()}\n\n")
    text_file.write(f"For censor data points (event = 0) : \n{cond_table[cond_table['event']==0]['TTE'].describe()}\n\n")
    # text_file.write(f"Curation distribution : \n{cond_table['curation'].value_counts()}\n\n")
    cond_table[cond_table['event']==1]['duration'].hist(bins=100)
    plt.savefig(f'{output_dir}_duration_event_hist_{cohort_org}.png')
    plt.show()
    cond_table[cond_table['event']==0]['duration'].hist(bins=100)
    plt.savefig(f'{output_dir}_duration_censor_hist_{cohort_org}.png')
    plt.show()
    cond_table[cond_table['event']==1]['TTE'].hist(bins=100)
    plt.savefig(f'{output_dir}_tte_event_hist_{cohort_org}.png')
    plt.show()
    cond_table[cond_table['event']==0]['TTE'].hist(bins=100)
    plt.savefig(f'{output_dir}_tte_censor_hist_{cohort_org}.png')
    plt.show()

    # if cohort_org in biomarker_schemas:
    # biomarker_df = get_biomarker(biomarker_schemas[cohort],schema,cond_table,cohort)
    # biomarker_df = get_all_biomarker(schema,cond_table,cohort_org)
    # # col_list = list(biomarker_df.columns)
    # # col_list.remove('chai_patient_id')
    # # biomarker_df[col_list] = biomarker_df[col_list].astype('category')
    # feature_dict['biomarker'] = list(biomarker_df.columns)
    # print(f"Biomarker : {biomarker_df.shape}, unique patients : {biomarker_df['chai_patient_id'].nunique()}")

    # imaging_df = get_imaging(schema, cond_table)
    # # col_list = list(imaging_df.columns)
    # # col_list.remove('chai_patient_id')
    # # imaging_df[col_list] = imaging_df[col_list].astype('category')
    # feature_dict['imaging'] = list(imaging_df.columns)
    # print(f"Imaging : {imaging_df.shape}, unique patients : {imaging_df['chai_patient_id'].nunique()}")

    # med_df = get_medication(schema, cond_table)
    # # col_list = list(med_df.columns)
    # # col_list.remove('chai_patient_id')
    # # med_df[col_list] = med_df[col_list].astype('category')
    # feature_dict['Med'] = list(med_df.columns)
    # print(f"med : {med_df.shape}, unique patients : {med_df['chai_patient_id'].nunique()}")
    
    # conmed_df = get_conmed(schema, cond_table)
    # # col_list = list(conmed_df.columns)
    # # col_list.remove('chai_patient_id')
    # # conmed_df[col_list] = conmed_df[col_list].astype('category')
    # feature_dict['ConMed'] = list(conmed_df.columns)
    # print(f"conmed : {conmed_df.shape}, unique patients : {conmed_df['chai_patient_id'].nunique()}")


    # stage_df = get_stage(schema, cond_table, cohort)
    # # col_list = list(stage_df.columns)
    # # col_list.remove('chai_patient_id')
    # # stage_df[col_list] = stage_df[col_list].astype('category')
    # feature_dict['Staging'] = list(stage_df.columns)
    # print(f"stage : {stage_df.shape}, unique patients : {stage_df['chai_patient_id'].nunique()}")

    # disease_statis_df = get_disease_status(schema, cond_table)
    # feature_dict['Disease Statis'] = list(disease_statis_df.columns)
    # print(f"disease_status : {disease_statis_df.shape}, unique patients : {disease_statis_df['chai_patient_id'].nunique()}")

    final_stat_df = get_labs(schema, cond_table)
    feature_dict['Labs'] = list(final_stat_df.columns)
    print(f"lab : {final_stat_df.shape}, unique patients : {final_stat_df['chai_patient_id'].nunique()}")

    final_stat_df = cond_table[['chai_patient_id','TTE','event']].merge(final_stat_df,on=['chai_patient_id'],how='left')

    # final_stat_df = final_stat_df.merge(med_df,on=['chai_patient_id'],how='left')
    # final_stat_df = final_stat_df.merge(conmed_df,on=['chai_patient_id'],how='left')
    #     final_stat_df = final_stat_df.merge(comorb_df,on='chai_patient_id',how='left')
    #     final_stat_df = final_stat_df.merge(surgery_df,on=['chai_patient_id', 'line_start'],how='left')

    #     final_stat_df = final_stat_df.merge(demographics_df,on=['chai_patient_id', 'line_start'],how='left')
    #     final_stat_df = final_stat_df.merge(radiation_df,on=['chai_patient_id'],how='left')
    # final_stat_df = final_stat_df.merge(stage_df,on=['chai_patient_id'],how='left')
    #     final_stat_df = final_stat_df.merge(tumor_exam_df,on=['chai_patient_id', 'line_start'],how='left')
    # final_stat_df = final_stat_df.merge(disease_statis_df,on=['chai_patient_id'],how='left')
    # if cohort_org in biomarker_schemas:
    # final_stat_df = final_stat_df.merge(biomarker_df,on=['chai_patient_id'],how='left')
    # final_stat_df = final_stat_df.merge(imaging_df,on=['chai_patient_id'],how='left')
    #     final_stat_df = final_stat_df.merge(censored,on=['chai_patient_id','line_start'],how='left')
    #     final_stat_df['line_duration'] = final_stat_df.apply(lambda x: -x['line_duration'] if x['censored']=='yes' else x['line_duration'], axis=1)
    # final_stat_df['censored'] = final_stat_df.apply(lambda x: False if x['censored']==True else True, axis=1)

    return final_stat_df

In [ ]:
# LOT_TABLES = dtu.s3_to_df('s3://c3-dev-model-factory/sai_vinod/sept_2023_pt360_delivery_unified_LOT_COT.json')['Patient360']['standard']
# def get_decoded_from_lot_table(lot_table):    
#     check = dtu.redshift_to_df(
#                             ''' SELECT version_of_model FROM {source_schema}'''.format(source_schema=lot_table))
#     secret_key = u'Kizqfoc8KapwPI4STeBfofaA_rV7nZNnbPSxLdT9KSM='
#     cipher_suite = Fernet(secret_key.encode('utf-8'))
#     a = check['version_of_model'].astype(str).values[1]  # check is LoT/CoT table dataframe
#     a = cipher_suite.decrypt((str.encode(a[2:-1])))
#     decoded = json.loads(a.decode("utf-8").replace("'", '"'))
#     schema = decoded['source_schema']
#     return decoded

In [ ]:
table_details = {'mds' : {'schema' : 'cdm_structured_202402',
                            'initial_dx_table' : 'model_factory_dev.initial_dx_date_cdm_structured_202402_v2_5_20240404_1034',
                            'lot_table_name' : 'line_of_therapy.mds_rbs_rwd360_cdm_structured_202402_lot_v6_7_20240404_1146'
                           }
                }

In [ ]:
%%time
dataframes = []
# patient_count = []
import time

for cohort in ['mds']:
    start = time.time() 
    print(cohort)
    
    schema = table_details[cohort]['schema']
    initial_dx_table = table_details[cohort]['initial_dx_table']
    lot_table_name = table_details[cohort]['lot_table_name']
    features = main(schema, initial_dx_table, lot_table_name, cohort)
    # patient_count.append(unique_patients)

    dataframes.append(features)
    print(time.time()-start)
# pickle.dump(dataframes, open('data/features_additional_lines_sept20.pkl', 'wb'))

In [ ]:
print('D')

In [ ]:
feature_list = ['Delta_mean_89247-1_0', 'Delta_min_89247-1_0', 'Delta_max_89247-1_0', 'Mean_89247-1_0', 'Median_89247-1_0', 
     'SD_89247-1_0', 'Minimum_89247-1_0', 'Maximum_89247-1_0', 'Skewness_89247-1_0', 'Kurtosis_89247-1_0',
     '25th_Percentile_89247-1_0', '75th_Percentile_89247-1_0', 'Range_89247-1_0', 'slope_89247-1_0', 'Delta_mean_89247-1_1',
     'Delta_min_89247-1_1', 'Delta_max_89247-1_1', 'Mean_89247-1_1', 'Median_89247-1_1', 'SD_89247-1_1', 'Minimum_89247-1_1',
     'Maximum_89247-1_1', 'Skewness_89247-1_1', 'Kurtosis_89247-1_1', '25th_Percentile_89247-1_1', '75th_Percentile_89247-1_1',
     'Range_89247-1_1', 'slope_89247-1_1', 'bevacizumab', 'paclitaxel', 'durvalumab', 'gemcitabine', 'sotorasib', 'docetaxel',
     'ramucirumab', 'pemetrexed', 'cisplatin', 'carboplatin', 'osimertinib', 'etoposide', 'atezolizumab', 'pembrolizumab',
     'nivolumab', 'ipilimumab', 'dexamethasone', 'stage', 'tstage', 'mstage', 'nstage', 'Complete therapeutic response_wind_0_45',
     'Partial therapeutic response_wind_0_45', 'Stable_wind_0_45', 'Tumor progression_wind_0_45',
     'Complete therapeutic response_wind_45_180', 'Partial therapeutic response_wind_45_180', 'Stable_wind_45_180',
     'Tumor progression_wind_45_180', 'tp53_mutation', 'CT', 'PET', 'MRI', 'Radioisotope', 'cohort']
len(feature_list)

In [ ]:
features.head()

In [ ]:
count_df = pd.DataFrame({'cohort':['Pancreas', 'Melanoma', 'Colorectal', 'Prostate', 'Bladder', 'Breast', 'Gastricesophagus', 'HCC', 'Lung', 'Renal'],'pat_count':[1326, 344, 268, 844, 472, 1144, 869, 133, 5900, 689]})

In [ ]:
import matplotlib.pyplot as plt
count_df = count_df.sort_values(by='pat_count',ascending=True)
plt.barh(count_df['cohort'],count_df['pat_count'])

In [ ]:
total_pat = sum(count_df['pat_count'])
print(f'Total patients : {total_pat}')
count_df['percentage'] = (count_df['pat_count']*100)/(total_pat)
count_df

In [ ]:
cohort = 'Renal'
decoded = get_decoded_from_lot_table(LOT_TABLES[cohort]['schema']+'.'+ LOT_TABLES[cohort]['table'])
schema = decoded['source_schema']
lot_table_name = LOT_TABLES[cohort]['schema']+'.'+LOT_TABLES[cohort]['table']
print(schema)
print(lot_table_name)
print(cohort)

In [ ]:
features,unique_patients = main(schema, lot_table_name, cohort)

In [ ]:
# dataframe_list = pickle.load(open('data/features_sept20.pkl', 'rb'))

final_data = pd.concat(dataframes).drop_duplicates(subset=['chai_patient_id']).reset_index().drop('index',axis=1)

In [ ]:
final_data[~final_data['chai_patient_id'].isin(temp_index_date['chai_patient_id'])]

In [ ]:
final_data.shape

In [ ]:
final_data['cohort'].unique()

In [ ]:
final_data = copy.deepcopy(final_stat_df)
final_data.shape

In [ ]:
final_data.head()

In [ ]:
final_data.shape

In [ ]:
final_data['chai_patient_id'].nunique()

In [ ]:
final_data1 = copy.deepcopy(final_data)

In [ ]:
cohort_pat_list = list(final_data[['chai_patient_id','Complete therapeutic response_wind_0_45',
 'Partial therapeutic response_wind_0_45',
 'Stable_wind_0_45',
 'Tumor progression_wind_0_45',
 'Complete therapeutic response_wind_45_180',
 'Partial therapeutic response_wind_45_180',
 'Stable_wind_45_180',
 'Tumor progression_wind_45_180',
 'tp53_mutation','CT',
 'PET','MRI','Radioisotope']].dropna(how='all')['chai_patient_id'])
final_data= final_data[final_data['chai_patient_id'].isin(cohort_pat_list)]
final_data = final_data.reset_index().drop('index',axis=1)
final_data.shape

In [ ]:
# cohort_pat_list = list(final_data.drop(['TTE','event'],axis=1).dropna(how='all')['chai_patient_id'])
# final_data= final_data[final_data['chai_patient_id'].isin(cohort_pat_list)]
# final_data = final_data.reset_index().drop('index',axis=1)
# final_data.shape

In [ ]:
mds_lab_df = pd.read_csv(f'{output_dir}/mds_labs_with_std_unite.csv')

In [ ]:
mds_lab_df.shape

In [ ]:
mds_lab_df.head()

In [ ]:
mds_lab_df['test_name_standard_code'].value_counts()

In [ ]:
mds_lab_df['test_name_standard_code'].value_counts()

In [ ]:
all_lab_codes = ['6690-2', '26464-8', '49498-9', '33256-9', '12227-5', '804-5', '718-7', '30313-1', '20509-6', '30350-3', '30351-1',
 '48725-6', '93846-4', '26499-4', '751-8', '753-4', '768-2', '30451-9', '777-3', '26515-7', '49497-1', '778-1', '26516-5',
 '53800-9', '74464-9', '13056-7', '24373-3', '2276-4', '489004']

lab_codes_c3_cnt_ab_100 = ['718-7','26464-8','26515-7','26499-4','777-3','751-8','6690-2','2276-4','753-4','12227-5','30451-9','30313-1',
                           '24373-3','768-2','20509-6','804-5','778-1','49497-1','30350-3','30351-1']
lab_codes_c0_cnt_ab_100 =['718-7','26515-7','26464-8','26499-4','2276-4','751-8','777-3','6690-2','30451-9','49497-1','24373-3',
                          '753-4','489004','49498-9']
lab_codes_common_ab_100 = ['718-7','26515-7','26464-8','26499-4','2276-4','751-8','777-3','6690-2','30451-9','49497-1','24373-3',
                          '753-4']
lab_codes_only_c0_ab_100 = ['489004', '49498-9']
lab_codes_only_c3_ab_100 = ['12227-5', '768-2', '778-1', '804-5','30350-3','30351-1']

In [ ]:
| 718-7                   ┆ 5783044 │#
│ 26515-7                 ┆ 5651132 │
│ 26464-8                 ┆ 5557732 │
│ 26499-4                 ┆ 4745147 │
│ 2276-4                  ┆ 905160  │
│ 751-8                   ┆ 695998  │
│ 777-3                   ┆ 235549  │
│ 6690-2                  ┆ 234353  │
│ 30451-9                 ┆ 103669  │
│ 49497-1                 ┆ 98237   │
│ 24373-3                 ┆ 40959   │#
│ 753-4                   ┆ 39742   │
│ 489004                  ┆ 2296    │#
│ 49498-9                 ┆ 517     │#
│ 12227-5                 ┆ 12      │
│ 20509-6                 ┆ 7       │
│ 48725-6                 ┆ 6       │
│ 33256-9                 ┆ 4       │
│ 30313-1                 ┆ 1       │
│ 778-1                   ┆ 1       │

In [ ]:
lab_dict = {'wbc':['49498-9','33256-9',],#26464-8,'6690-2','12227-5','804-5'
            'hemoglobin_in_blood':['48725-6','93846-4'],#'718-7','30313-1','20509-6','30350-3','30351-1'
            'neutrophils_count':[],#'26499-4','751-8','753-4','30451-9','768-2'
            'platelets':['26516-5','53800-9','74464-9','13056-7'], #26515-7,'777-3','778-1','49497-1'
            'ferritin':['489004']#'2276-4','24373-3'
           }


In [ ]:
from mds_labs_model2 import run_labs
run_labs_str = "','".join(run_labs)
patient_test_df = dtu.redshift_to_df('''SELECT 
chai_patient_id,curation_indicator,test_name_standard_name,test_name_standard_code,
    test_date,test_value_numeric_standard,test_unit_standard_name FROM {schema}.patient_test
    WHERE ((test_name_standard_code IN ('{run_labs_str}')))'''.format(schema=schema, run_labs_str=run_labs_str))

In [ ]:
patient_test_df['test_name_standard_code'].value_counts()

In [ ]:
x_train.head()

In [ ]:
final_data.describe()

In [ ]:
hemoglobin_718_7_col = [x for x in x_train.columns if 'wbc_26464-8' in x]

In [ ]:
import seaborn as sns

fig, ax = plt.subplots()
fig.set_figheight(20)
fig.set_figwidth(20)
#plotting the heatmap for correlation
ax = sns.heatmap(final_data[hemoglobin_718_7_col+['event','TTE']].corr(), annot=True, linewidths=.5)

In [ ]:
x_train.shape

In [ ]:
# from xgbse import XGBSEKaplanNeighbors
# from xgbse._kaplan_neighbors import DEFAULT_PARAMS
from xgbse.metrics import concordance_index
from xgbse import XGBSEDebiasedBCE
from xgbse.converters import convert_to_structured
import os
import time
# medication_df = pickle.load(open('data/medication_df.pkl', 'rb'))

# drug_list = pd.read_csv('data/cancer_drugs_public_V13_0.csv')
# chemo_drug_list = drug_list[drug_list['primary_category']=='Chemotherapy']
# chemo = []
# for col in medication_df.columns:
#     if col in chemo_drug_list['active_ingredient'].tolist():
#         print(col)
#         chemo.append(col)
# medication_df = medication_df[chemo + ['chai_patient_id', 'line_start']]

In [ ]:
# final_data1[['Complete therapeutic response_wind_0_45',
#  'Partial therapeutic response_wind_0_45',
#  'Stable_wind_0_45',
#  'Tumor progression_wind_0_45',
#  'Complete therapeutic response_wind_45_180',
#  'Partial therapeutic response_wind_45_180',
#  'Stable_wind_45_180',
#  'Tumor progression_wind_45_180']].dropna(how='all').shape

In [ ]:
# final_data[['Complete therapeutic response_wind_0_45',
#  'Partial therapeutic response_wind_0_45',
#  'Stable_wind_0_45',
#  'Tumor progression_wind_0_45',
#  'Complete therapeutic response_wind_45_180',
#  'Partial therapeutic response_wind_45_180',
#  'Stable_wind_45_180',
#  'Tumor progression_wind_45_180','tp53_mutation','CT',
#  'PET',
#  'MRI',
#  'Radioisotope']].dropna(how='all').shape

In [ ]:
# new_labs = pickle.load(open('mc/lab_features_sept_14_0.pkl', 'rb'))
# dataframe_list = pickle.load(open('data/features_sept20.pkl', 'rb'))
# final_data = pd.concat(dataframe_list).drop_duplicates().drop(columns = [np.nan], axis = 1)
# final_data = pd.concat([final_data, final_data1]).drop_duplicates()
# cohort_df = pd.get_dummies(final_data, columns = ['cohort', 'ethnicity', 'gender', 'race'], prefix = None)
# cohort_df = cohort_df.drop(np.nan, axis = 1)
# drop_these = []
# for col in list(cohort_df.columns):
#     for s in ['Mean', 'Median_', 'SD_', 'Minimum_', 'slope',
#               'Maximum_', 'Skewness_', 'Kurtosis_', '25th_Percentile_', '75th_Percentile_', 'Range_']:
#         if col.startswith(s):
#             drop_these.append(col)
# cohort_df = cohort_df.drop(columns = drop_these, axis = 1)
# cohort_df = cohort_df.merge(new_labs, on = ['chai_patient_id', 'line_start'], how = 'left')
# X = final_data.drop(['chai_patient_id','last_available_date','line_start','line_end','line_duration','random_point','line_number','target_duration',
#                      'censored', 'stage_date'],axis=1)
X = final_data.drop(['chai_patient_id','TTE','event'],axis=1)
# X = X[feature_list]

# Y = cohort_df[['line_duration']]

regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=348)
# y_train['event'] = y_train.apply(lambda x: 1 if x['line_duration']>0 else 0, axis = 1)
# y_test['event'] = y_test.apply(lambda x: 1 if x['line_duration']>0 else 0, axis = 1)

# y_train_np = y_train.to_records(index=False)
# y_train_np = np.array(y_train_np, dtype = y_train_np.dtype.descr)

# y_test_np = y_test.to_records(index = False)
# y_test_np = np.array(y_test_np, dtype = y_test_np.dtype.descr)



# print("======================")
# # loop to show different scale results
# m = []
# for scale in [1.0]:
#     start = time.time()
#     DEFAULT_PARAMS['aft_loss_distribution_scale'] = scale
#     xgbse_model = XGBSEKaplanNeighbors(DEFAULT_PARAMS, n_neighbors=30)
#     xgbse_model.fit(X_train, y_train_np)
#     m.append(xgbse_model)
#     preds = xgbse_model.predict(X_test)
#     preds_train = xgbse_model.predict(X_train)
#     cind_test = concordance_index(y_test_np, preds)
#     cind_train = concordance_index(y_train_np, preds_train)
#     print(f"aft_loss_distribution_scale: {scale}")
#     print(f"C-index, train: {cind_train:.3f}")
#     print(f"C-index, test: {cind_test:.3f}")
#     #print(f"Average probability of survival: {avg_probs}")
#     print("----", time.time() - start)
# # pickle.dump(m, open('models/xgbse_models_nlp_sept20_additional_lines.pkl', 'wb'))

In [ ]:
# final_data['censored']=final_data['censored'].apply(lambda x:False if x==True else True)

In [ ]:
y = convert_to_structured((final_data['TTE']),final_data['event'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=938)

In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
text_file.write(f"x_train shape : {x_train.shape}\n,x_test shape : {x_test.shape}\n,y_train shape : {y_train.shape}\n,y_test shape : {y_test.shape}\n\n")

In [ ]:
# print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
x_train.dtypes

In [ ]:
x_train.describe()

In [ ]:
# column_list = list(x_train.columns)
# len(column_list)

In [ ]:
# with open(f'data_backup/column_list_11_28_2023.pkl','wb') as f:
#     pickle.dump(column_list,f)

In [ ]:
%%time
PARAMS_XGB_AFT = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'aft_loss_distribution': 'normal',
    'aft_loss_distribution_scale': 1.0,
    'tree_method': 'hist',
    'learning_rate': 0.005,
    'max_depth': 16,
    'booster':'dart',
    'subsample': 0.8,
    'min_child_weight': 30,
    'colsample_bynode':0.8
}

xgbse_model = XGBSEDebiasedBCE(PARAMS_XGB_AFT)
xgbse_model.fit(x_train,y_train,time_bins=np.array([60,90,120,150]))

In [ ]:
# x_train_temp = x_train.fillna(-999)

In [ ]:
# from sklearn.feature_selection import RFE

# rfe_model = RFE(xgbse_model,n_features_to_select=100)
# rfe_model.fit(x_train_temp,y_train)

In [ ]:
x_test.head()

In [ ]:
final_data.head()

In [ ]:
cond_list = []
file_list = os.listdir(output_dir)
file_list = [x for x in file_list if 'cond' in x]

In [ ]:
curation_1_list = []
curation_0_list = []
for file in file_list:
    temp_cond = pd.read_csv(f'{output_dir}{file}')
    temp_cond_0 = temp_cond[temp_cond['curation']==0]
    temp_cond_1 = temp_cond[temp_cond['curation']==1]
    curation_0_list+=list(temp_cond_0['chai_patient_id'].unique())
    curation_1_list+=list(temp_cond_1['chai_patient_id'].unique())
    
curation_1_list = list(set(curation_1_list))
curation_0_list = list(set(curation_0_list))

In [ ]:
x_test.shape

In [ ]:
y_test

In [ ]:
final_data_0 = final_data[final_data['chai_patient_id'].isin(curation_0_list)]
final_data_0 = final_data_0[final_data_0['Unnamed: 0'].isin(x_test['Unnamed: 0'].unique())]
X = final_data_0.drop(['chai_patient_id','TTE','event','Unnamed: 0'],axis=1)
y = convert_to_structured((final_data_0['TTE']),final_data_0['event'])

X.shape

In [ ]:
concordance_index(y, xgbse_model.predict(X))

In [ ]:
train_cindex = concordance_index(y_train, xgbse_model.predict(x_train))
test_cindex = concordance_index(y_test, xgbse_model.predict(x_test))

In [ ]:
print(f'Train c-index : {train_cindex}')
print(f'Test c-index : {test_cindex}')
text_file.write(f'Train c-index : {train_cindex} \n')
text_file.write(f'Test c-index : {test_cindex} \n')

In [ ]:
concordance_index(y, xgbse_model.predict(X))

In [ ]:
pred = []
pred.append(xgbse_model.predict(x_test))
# pred.append(xgbse_model.predict(X))
    
event_prob_df = 1-pred[0]
# event_prob_df.to_csv(f'data_backup/all_feature_prob_{today_date}.csv')
text_file.write('Number of patients above 0.8 prediction probability\n')
selected_pt_list = []
for col in event_prob_df.columns:
    selected_df = event_prob_df[event_prob_df[col]>=0.8]
    main_index = list(x_test.reset_index().iloc[list(selected_df.index)]['index'])
    selected_pt_list.append(list(final_data.iloc[main_index]['chai_patient_id']))
    print(f'{col} : {selected_df.shape}')
    text_file.write(f'For window {col} : {selected_df.shape[0]}')

In [ ]:
import matplotlib.pyplot as plt
for i in [60,90,120,150]:
    print(f'window : {i}')
    event_prob_df[i].hist(bins=100)
    plt.savefig(f'{output_dir}predicted_prob_distribution_{i}.png')
    plt.show()

In [ ]:
output_dir

In [ ]:
text_file.write('Statistics of predicted probabbility\n')
for i in [60,90,120,150]:
    text_file.write(f'stats for window {i} : {event_prob_df[i].describe()}\n')

In [ ]:
selcted_patient_info = pd.DataFrame({'chai_patient_id':slelected_pt_list[3]})
window =[60,90,120,150]
for i in range(len(slelected_pt_list)):
    selcted_patient_info[window[i]] = selcted_patient_info['chai_patient_id'].apply(lambda x:True if x in slelected_pt_list[i] else False)

selcted_patient_info = selcted_patient_info.merge(final_data,on='chai_patient_id',how='inner') 

In [ ]:
import pickle
import joblib
date = '05_31_2024'
event_prob_df.to_csv(f'{output_dir}event_prob_df_{date}.csv')
x_train.to_csv(f'{output_dir}x_train_{date}.csv')
x_test.to_csv(f'{output_dir}x_test_{date}.csv')
np.save(f'{output_dir}y_train_{date}.npy',y_train)
np.save(f'{output_dir}y_test_{date}.npy',y_test)
final_data.to_csv(f'{output_dir}final_data_{date}.csv')
with open(f'{output_dir}model_{date}.pkl','wb') as f:
    pickle.dump(xgbse_model,f)
joblib.dump(xgbse_model,f'{output_dir}model_{date}.joblib')

In [ ]:
output_dir

In [ ]:
import pickle
import joblib
date = '05_24_2024'

# event_prob_df = pd.read_csv('./data_backup_model2/experiment_m2_lung_old_cohort_2024-02-29/event_prob_df_02_29_2024.csv')
# event_prob_df = pd.read_csv(f'{output_dir}event_prob_df_{date}.csv')
x_train = pd.read_csv(f'{output_dir}x_train_{date}.csv')
x_test = pd.read_csv(f'{output_dir}x_test_{date}.csv')
y_train = np.load(f'{output_dir}y_train_{date}.npy')
y_test = np.load(f'{output_dir}y_test_{date}.npy')
final_data = pd.read_csv(f'{output_dir}final_data_{date}.csv')

# with open(f'{output_dir}model_{date}.pkl','rb') as f:
#     xgbse_model = pickle.load(f) 

In [ ]:
# event_prob_df = event_prob_df.drop('Unnamed: 0',axis=1)
x_train = x_train.drop('Unnamed: 0',axis=1)
x_test = x_test.drop('Unnamed: 0',axis=1)
final_data = final_data.drop('Unnamed: 0',axis=1)

In [ ]:
x_train.describe()

In [ ]:
x_train.shape

In [ ]:
final_data[final_data['event']==1]['TTE'].hist(bins=100)

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import PrecisionRecallDisplay,ConfusionMatrixDisplay,confusion_matrix,roc_auc_score
import matplotlib.pyplot as plt
combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_data[['chai_patient_id','TTE','event']].reset_index(),on='index',how='inner')

# combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_stat_df.reset_index()[['index','chai_patient_id','label','final_selection']],on='index',how='inner')
for i in [60,90,120,150]:
    combined_test_info_df[f'final_selection_{i}'] = combined_test_info_df.apply(lambda x: 0 if x['TTE']>i else x['event'],axis=1)
    print(combined_test_info_df[f'final_selection_{i}'].value_counts())
    print(f"ROC AUC Score : {roc_auc_score(combined_test_info_df[f'final_selection_{i}'], combined_test_info_df[i])}")
    text_file.write(f'Label counts for window : {i} \n')
    text_file.write(f"{combined_test_info_df[f'final_selection_{i}'].value_counts()} \n\n")
    PrecisionRecallDisplay.from_predictions(combined_test_info_df[f'final_selection_{i}'], combined_test_info_df[i], name="PA_model", plot_chance_level=True)
    plt.title(f'window : {i}')
    plt.legend(loc=(0.5, 0.8))
    plt.savefig(f'{output_dir}/PRC_Window_{i}.png')
    plt.plot()

In [ ]:
feature_importance_df = pd.DataFrame(xgbse_model.feature_importances_.items(),columns=['feature','importance'])

In [ ]:
feature_importance_df = feature_importance_df.sort_values(by='importance',ascending=False)
feature_importance_df.shape

In [ ]:
ft_used = list(set(feature_importance_df['feature']))
ft_used_df = pd.DataFrame({'feature':ft_used})

ft_used_df['window'] = ft_used_df['feature'].apply(lambda x:'_'.join(x.split('_')[-2:]) if (len(x.split('_'))>1) else None)
ft_used_df['lab'] = ft_used_df['feature'].apply(lambda x:'_'.join(x.split('_')[-4:-2]) if (len(x.split('_'))>1) else x)
ft_used_df['stats_type'] = ft_used_df['feature'].apply(lambda x:'_'.join(x.split('_')[:-4]) if (len(x.split('_'))>1) else None)

In [ ]:
ft_used_df['lab'].value_counts()

In [ ]:
ft_not_used = list(set(x_train.columns)-set(feature_importance_df['feature']))

In [ ]:
ft_not_used_df = pd.DataFrame({'feature':ft_not_used})

In [ ]:
ft_not_used_df['window'] = ft_not_used_df['feature'].apply(lambda x:'_'.join(x.split('_')[-2:]) if (len(x.split('_'))>1) else None)
ft_not_used_df['lab'] = ft_not_used_df['feature'].apply(lambda x:'_'.join(x.split('_')[-4:-2]) if (len(x.split('_'))>1) else x)
ft_not_used_df['stats_type'] = ft_not_used_df['feature'].apply(lambda x:'_'.join(x.split('_')[:-4]) if (len(x.split('_'))>1) else None)

In [ ]:
ft_not_used_df

In [ ]:
ft_not_used_df['lab'].value_counts()

In [ ]:
['carboplatin','pembrolizumab','pemetrexed','paclitaxel','durvalumab','osimertinib','nivolumab', 'docetaxel', 'ipilimumab','gemcitabine',
 'ramucirumab','etoposide','atezolizumab','bevacizumab','cisplatin','sotorasib','erlotinib','afatinib','gefitinib'
,'dacomitinib','amivantamab','mobocertinib','necitumumab','adagrasib','crizotinib','ceritinib','alectinib','brigatinib','lorlatinib']
29

In [ ]:
feature_importance_df =feature_importance_df.sort_values(by='importance',ascending=False)
combine_feature_importance_df_subset = feature_importance_df[:20]
combine_feature_importance_df_subset = combine_feature_importance_df_subset.sort_values(by='importance',ascending=True)

In [ ]:
feature_importance_df.iloc[100:160]

In [ ]:
f, ax = plt.subplots(figsize=(20,10))
plt.barh(combine_feature_importance_df_subset['feature'],combine_feature_importance_df_subset['importance'])
# plt.xticks(range(len(combine_feature_importance_df_subset['feature'])), combine_feature_importance_df_subset['average'],rotation='vertical')
plt.savefig(f'{output_dir}/importance_plot.png',bbox_inches = 'tight')
plt.show()

In [ ]:
combined_test_info_df.head()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
import matplotlib.pyplot as plt
# combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_stat_df.reset_index()[['index','chai_patient_id','label','final_selection']],on='index',how='inner')

for window in [60,90,120,150]:
    for thresold in [0.1]:
        combined_test_info_df[f'pred_{thresold}'] = combined_test_info_df[window].apply(lambda x:x>thresold)
        cm = confusion_matrix(combined_test_info_df[f'final_selection_{window}'],combined_test_info_df[f'pred_{thresold}'])
        disp = ConfusionMatrixDisplay(cm)
        disp.plot()
        plt.title(f'Thresold : {thresold}, Window : {window}')
        plt.savefig(f'{output_dir}/CF_thresold_{thresold}_Window_{window}.png')

In [ ]:
for window in [60,90,120,150]:
    combined_test_info_df[combined_test_info_df[f'final_selection_{window}']==True][window].plot.kde()
    combined_test_info_df[combined_test_info_df[f'final_selection_{window}']==False][window].plot.kde()
    plt.legend(['Event','Censored'])
    plt.title(f'Prediction window : {window}')
    plt.savefig(f'{output_dir}/pred_prob_classwise_kde_window_{window}.png',bbox_inches = 'tight')
    plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
# combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_stat_df.reset_index()[['index','chai_patient_id','label','final_selection']],on='index',how='inner')
metric_list = []
for window in [60,90,120,150]:
    window_df = pd.DataFrame({'window':[window]})
    for thresold in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        combined_test_info_df[f'pred_{thresold}'] = combined_test_info_df[window].apply(lambda x:x>thresold)
        cm = confusion_matrix(combined_test_info_df[f'final_selection_{i}'],combined_test_info_df[f'pred_{thresold}'])
        tn,fp,fn,tp = cm[0,0],cm[0,1],cm[1,0],cm[1,1]
        p1 = tp/(tp+fp)
        r1 = tp/(tp+fn)
        f1 = 2*p1*r1/(p1+r1)
        p0 = tn/(tn+fn)
        r0 = tn/(tn+fp)
        f0 = 2*p0*r0/(p0+r0)
        window_df = pd.concat([window_df,pd.DataFrame({
            f'tn_{thresold}' : [tn],
            f'fp_{thresold}' : [fp],
            f'fn_{thresold}' : [fn],
            f'tp_{thresold}' : [tp],
            f'p1_{thresold}' : [p1],
            f'r1_{thresold}' : [r1],
            f'f1_{thresold}' : [f1],
            f'p0_{thresold}' : [p0],
            f'r0_{thresold}' : [r0],
            f'f0_{thresold}' : [f0],
        })],axis=1)
    metric_list.append(window_df)
metric_df = pd.concat(metric_list)

In [ ]:
metric_dfß

In [ ]:
text_file.close()

In [ ]:
import shap
x_train_sample = x_train.sample(n=200,random_state=123).astype('float64')
explainer = shap.Explainer(xgbse_model.predict,x_train_sample)
shap_values = explainer(x_train_sample,max_evals='auto')

In [ ]:
shap.plots.beeswarm(shap_values[:,:,1],max_display=100,show=False)
plt.savefig(f'{output_dir}/shap_plot.png',bbox_inches = 'tight')
plt.show()